In [1]:
%cd ..

/run/media/nazif/2F946E411BA61D49/thesis


In [4]:
from scripts.utils_latest import *


import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.colors as mcolors
import seaborn as sns
pd.set_option('display.max_columns', None)


In [8]:
df = pd.read_csv("results/1_generate_data.csv")
clash_df = clash()
df.head()

,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,id,clash_mrna_start,clash_mrna_end,true_seed_type,true_num_basepairs,true_seed_basepairs,true_energy,mirna_sequence,mirna_accession,enst,ensg,alignment_string,pred_num_basepairs,pred_seed_basepairs,bp_difference,seed_bp_difference,mre_start,mre_end,full_mrna_seq,mre_region
0,23,46,-27.7,1,21,)))))))))))))))))))).,0727A-1038930_1,1791,1890,noncanonical_seed,20,6,-25.1,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340828,ENSG00000113328,1111111111111111111100,20,6,0,0,23,44,ATTTGTATCTACGATAAAAATTTTTATACAGAACCTACTGCCTCAA...,TTTATACAGAACCTACTGCCTC
1,12,31,-26.2,1,19,))))))))).)))))))).,L1HS-1112536_1,3857,3928,9-mer,17,6,-24.4,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000343455,ENSG00000100697,1111111110111111110000,17,6,0,0,12,33,CAGGAAATACCCGTGCAACCAACTACCTCATATTCCATTCAGAATT...,CGTGCAACCAACTACCTCATAT
2,4,25,-23.6,1,22,.)))))).))))))))))))).,L2HS-818542_2,2385,2434,noncanonical_seed,19,6,-22.2,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000436639,ENSG00000080546,0111111011111111111110,19,6,0,0,4,25,ACCCGCTATATGACCTGATGCCTTTCCTTCATTAAAGATGATTCTG...,CGCTATATGACCTGATGCCTTT
3,4,25,-23.8,1,22,)))))))...)))))))))))),L2HS-1161339_2,6570,6623,noncanonical_seed,18,6,-22.1,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000282516,ENSG00000164190,1111111000111111111111,19,6,1,0,4,25,CAATGACTATGCAACCATACCTTACCACTAAATGTAGTACGCAAAA...,TGACTATGCAACCATACCTTAC
4,3,25,-24.0,1,22,))))))))))))))...)))).,L2-407944_2,1164,1208,noncanonical_seed,18,6,-21.9,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ENST00000340139,ENSG00000138785,1111111111111100011110,18,6,0,0,3,24,AATACTGGGAAACCTGCTACTTCGTCAGCTAACCAGAAACCTGTG,TACTGGGAAACCTGCTACTTCG


In [5]:
negative_df = df[["mre_region", "mirna_sequence", "mirna_accession"]]

# shuffle mirna sequences using levenstein distance
for i, row in negative_df.iterrows():
    original_string = row['mirna_sequence']
    # exclude the original string from the column
    column = negative_df['mirna_sequence'].drop(i)
    new_string = find_most_different_string(original_string, column)
    negative_df.at[i, 'shuffled_sequence'] = new_string

/tmp/ipykernel_43893/4074009060.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_df.at[i, 'shuffled_sequence'] = new_string


In [6]:
# renaming cols for convention
rename_dict = {"mrna_sequence": "mre_region", "mirna_sequence": "old_mirna_sequence",
               "mirna_accession": "old_mirna_accession", "shuffled_sequence": "mirna_sequence"}
negative_df = negative_df.rename(columns=rename_dict)

In [9]:
# adding accessions for new mirna sequences
accession_dict = clash_df.set_index('mirna_sequence')[
    'mirna_accession'].to_dict()
negative_df['mirna_accession'] = negative_df['mirna_sequence'].map(
    accession_dict)

In [11]:
negative_df = negative_df.rename(columns={"mre_region": "mrna_sequence"})


# finding matches on negative data
negative_df_results = find_matches_with_rnaduplex(negative_df)

# adding relevant columns
negative_df_results["mirna_sequence"] = negative_df["mirna_sequence"]
negative_df_results["mre_region"] = negative_df["mrna_sequence"]
negative_df_results["mirna_accession"] = negative_df["mirna_accession"]
negative_df_results["enst"] = clash_df["enst"]



In [ ]:
# export to csv
negative_df_results.to_csv("results/3_negative_data.csv", index=False)